# Homework 4: CIFAR-10
Applied Neural Networks

## Create a CNN to classify CIFAR-10
Use the [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) dataset.

You can utilize `keras.datasets.cifar10.load_data()` to import the dataset as `numpy ndarrays` or `tfds.load('cifar10')` to import the dataset as `tf.data.Dataset`.

**You must utilize transfer learning and data augmentation.** Your grade will depend on the quality of your best model. Discuss the different models you tried. *Include all model parameters, accuracy, a confusion matrix, and sample misclassified images.*

I suggest using data pipelines to avoid a RAM resource exhaustion error. They can be implemented fairly easily in one of two ways: 
  1. Utilize a data_augmentation function and Dataset.map(data_augmentation) to augment your training tf.data.Dataset (will require `tf.data.Dataset`s), or
  2. Utilize Keras ImageDataGenerator, using the ImageDataGenerator to also do all preprocessing necessary and batch images (will require `np ndarray`s).

In [26]:
# using keras.datasets.cifar10.load_data() and Keras ImageDataGenerator

# imports
import numpy as np
import pandas as pd
from sklearn.utils.multiclass import unique_labels
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import itertools
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import tensorflow
from tensorflow import keras

from keras import Sequential
# Models imported for transfer learing
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications import ResNet50

# some of the import functions only work by putting tensorflow in them. Not sure why
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Flatten, Dense, BatchNormalization, Activation, Dropout
from tensorflow.keras.utils import to_categorical

In [15]:
from keras.datasets import cifar10

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170508288/170498071 [==============================] - 2s 0us/step


In [16]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=.3)

In [17]:
X_train.shape, y_train.shape

((35000, 32, 32, 3), (35000, 1))

In [18]:
X_val.shape, y_val.shape

((15000, 32, 32, 3), (15000, 1))

In [19]:
X_test.shape, y_test.shape

((10000, 32, 32, 3), (10000, 1))

In [20]:
# 10 labels so the y shape will go from 1 to 10
# https://www.geeksforgeeks.org/python-keras-keras-utils-to_categorical/
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

In [21]:
y_train.shape, y_val.shape, y_test.shape

((35000, 10), (15000, 10), (10000, 10))

In [25]:
# Data Augmentation using the ImageDataGenerator
train_generator = ImageDataGenerator(rotation_range=2,
                                     horizontal_flip=True,
                                     zoom_range=0.1)
val_generator = ImageDataGenerator(rotation_range=2,
                                   horizontal_flip=True,
                                   zoom_range=0.1)
test_generator = ImageDataGenerator(rotation_range=2,
                                    horizontal_flip=True,
                                    zoom_range=0.1)

# fit the generators
train_generator.fit(X_train)
val_generator.fit(X_val)
test_generator.fit(X_test)

In [48]:
# Adjustable learning rate using keras callbacks
# error: monitor needed to be changed from val_accuracy to accuracy?
learning_rate_reduce = ReduceLROnPlateau(monitor='val_accuracy',
                                         factor=0.01,
                                         patience=3,
                                         min_lr=1e-5)

In [27]:
# first model uses the VGG19 model specifically designed for image recognition
model_1 = VGG19(include_top=False,
                weights='imagenet',
                input_shape=(32,32,3),
                classes=y_train.shape[1])

80150528/80134624 [==============================] - 1s 0us/step


In [50]:
# Add finishing layers to model 1
model = Sequential()
model.add(model_1)
model.add(Flatten())
model.add(Dense(1024, activation=('relu'), input_dim=512))
model.add(Dense(512, activation=('relu')))
model.add(Dense(256, activation=('relu')))
model.add(Dense(128, activation=('relu')))
model.add(Dense(10, activation=('softmax')))

model.compile(optimizer=SGD(lr=.001,
                            momentum=.9,
                            nesterov=False),
              loss='categorical_crossentropy',
              metrics=[tensorflow.keras.metrics.Accuracy(name="accuracy", dtype=None)])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [51]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 1, 1, 512)         20024384  
                                                                 
 flatten_6 (Flatten)         (None, 512)               0         
                                                                 
 dense_25 (Dense)            (None, 1024)              525312    
                                                                 
 dense_26 (Dense)            (None, 512)               524800    
                                                                 
 dense_27 (Dense)            (None, 256)               131328    
                                                                 
 dense_28 (Dense)            (None, 128)               32896     
                                                                 
 dense_29 (Dense)            (None, 10)               

batch_size= 100
epochs=50

learn_rate=.001

sgd=SGD(lr=learn_rate,momentum=.9,nesterov=False)
adam=Adam(lr=learn_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

In [52]:
# The data generator can be removed from the validation set in order to see
# different results.
# steps per epoch error fix
# https://stackoverflow.com/questions/59864408/tensorflowyour-input-ran-out-of-data
history = model.fit_generator(train_generator.flow(X_train, y_train, batch_size=128),
                    epochs=50,
                    steps_per_epoch=len(X_train)//128,
                    validation_data=val_generator.flow(X_val, y_val, batch_size=128),
                    #validation_data=(X_val, y_val),
                    validation_steps=250,
                    callbacks=[learning_rate_reduce],
                    #callbacks=[keras.callbacks.EarlyStopping(patience=5)],
                    verbose=1)

Epoch 1/50


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  del sys.path[0]


273/273 [==============================] - 53s 188ms/step - loss: 0.3970 - accuracy: 2.8103e-04 - val_loss: 0.4948 - val_accuracy: 5.4000e-04 - lr: 0.0010
Epoch 2/50
273/273 [==============================] - ETA: 0s - loss: 0.2913 - accuracy: 4.2154e-04

KeyboardInterrupt: ignored

In [28]:
# Second model uses ResNet50, which is a deeper and more compelx model than the previous VGG19
model_2 = ResNet50(include_top=False,
                   weights='imagenet',
                   input_shape=(32,32,3),
                   classes=y_train.shape[1])

94781440/94765736 [==============================] - 1s 0us/step
